## We are implementing Reinforce in DOOM env

Remember that this is a complementary notebook-guide to the Medium post that probably brought you here, so the explanation you may be seeking is there.

![](images/doom_env.png)

#### Imports block

Here we import the modules used on our training. Numpy is used for generating the possible actions list and Matplotlib for some printing at the end. VizDoom and Torch and TorchVision are our main modules.

In [1]:
from vizdoom import *

import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from torch.autograd import Variable

#### Defining our device usage and path to save the network

You have the possibility of using it on Intel MKL-powered PyTorch for CPU (as I did, as we have low-cost with reasonable performance) or on GPU if you have one and want to try it. We also set the random seed and use deterministic-only algorithms on CUDA (if used, for reproducebility).

In [3]:
use_cuda = torch.cuda.is_available() 
device = torch.device('cuda' if use_cuda else 'cpu')
#use_cuda = False
#device = torch.device('cpu')
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor


torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

if use_cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#### We here set the game assetes and load its configuration files.
It will popup a game window, but with no move as we are not yet performing actions on it. We also generate the possible actions list here.

In [4]:
#Doom game assets: game env creation and frame resize functiom
game = DoomGame()
game.load_config("health_gathering.cfg")
game.set_doom_scenario_path("health_gathering.wad")
game.set_seed(42)
game.init()
doom_actions = np.identity(3, dtype=int).tolist()

#### We also must create a Frame Stacker class

Here is our FrameStacker class. we use it to approach the non-Markov propriety issue on our frame-based env. It stacks the last 4 frames and return it with the input shape of our neural network, preprocessing it on the middle of the way.


In [5]:

class FrameStacker:
    def __init__(self):
        """
        We can set the memory size here.
        Our memory is a deque and, on each stack, it concatenates the frames in memory along the axis 0
        We also have a transformer from torch that handles the resizing.
        """
        self.memory_size = 4
        self.memory = deque(maxlen=self.memory_size)
        self.reset()
        self.transformer = T.Compose([T.ToPILImage(),
                                      T.Resize((84,84)),
                                      T.ToTensor()])
    def reset(self):
        """
        by feeding the deque with zero-tensors we restart the memory.
        """
        for i in range(4):
            self.memory.append(torch.zeros(1, 84, 84).to(device))
            
    def preprocess_frame(self, frame):
        """
        here we handle the cutting and flowing the frame through the transformer
        """
        frame = frame[80:,:]
        frame = self.transformer(frame)
        return frame.to(device)/255

    def stack(self, frame):
        """
        our stack method preprocesses the state and returns it stacked.
        """
        frame = self.preprocess_frame(frame)
        self.memory.append(frame)
        return torch.cat(tuple(self.memory), dim=0)

#### Here we define the architecture of our Policy Network.

Notice that we use many Tanh activation functions in order to bring more variance to the learning process, and also use BatchNorms to speed up training. You will be able to see it on TensorBoard if you feel like it.

![](images/network_architecture.png)

In [6]:
class PolicyNetwork(nn.Module):
    def __init__(self, lr):
        """
        We've put Tanh as activation in order to introduce variance on the learning.
        Adam optimizer was the best performant.
        I encourage you to try other architectures, optimizers and hyperparameters
        """
        super(PolicyNetwork, self).__init__()
        
        self.num_actions = 3
        
        self.conv_net = nn.Sequential(nn.Conv2d(in_channels=4, out_channels=32, kernel_size=8, stride=4),
                   nn.BatchNorm2d(32),
                   nn.ELU(True),
                   nn.Conv2d(32, 64, kernel_size=4, stride=2),
                   nn.BatchNorm2d(64),
                   nn.ELU(True),
                   nn.Conv2d(64, 128, kernel_size=4, stride=2 ),
                   nn.BatchNorm2d(128),
                   nn.ReLU(True))
        
        self.linear = nn.Sequential(nn.Linear(1152, 512),
                                    nn.Tanh(),
                                    nn.Linear(512, 512),
                                    nn.Tanh(),
                                    nn.Linear(512, self.num_actions),
                                    nn.Tanh(),)
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

        
    def forward(self, state_stack):
        """
        simple feedforward method
        """
        x = self.conv_net(state_stack)
        x = x.view(x.size(0), -1)
        x = F.softmax(self.linear(x), dim=1)
        return x

    def get_action(self, state):
        state = state.float().unsqueeze(0)
        probs = self.forward(Variable(state))
        
        #we've decided to use stochastic action learning in order to introduce variance in the learning
        distribution = torch.distributions.categorical.Categorical(probs = probs.detach())
        highest_prob_action = distribution.sample()
        
        
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action])
        
        #it returns the useful values for acting and optimizing
        return highest_prob_action, log_prob

#### Here is our Update Policy function

We get the normalized and discounted rewards, use it as labels with the log-softmax score function of the outputs of our network. The stochastic action leraning lets us learn the small probability actions too, which is good for training, as there are more situations for us to expose the model to.

In [7]:
def update_policy(policy_network, rewards, log_probs):
    discounted_rewards = []

    for t in range(len(rewards)):
        Gt = 0
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw += 1
        discounted_rewards.append(Gt)

    discounted_rewards = torch.tensor(discounted_rewards, device=device)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std())
    
    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    
    policy_network.optimizer.zero_grad()
    policy_gradient_ = torch.stack(policy_gradient[t:]).sum()
    policy_gradient_.backward(retain_graph=True)
    policy_network.optimizer.step()

#### Finally we set up TensorBoard and some global variables

After running this cell, you can seek the model architecture (and later its learning) on TensorBoard, with the command:

```
tensorboard --logdir runs
```

If you are running this for the second time you may want to delete your TensorBoard previous records:
```
rm -r runs
```

In [8]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(flush_secs = 40)

#here we set the global variables
GAMMA = .95
EPISODES = 5000
learning_rate = 0.02

#our net and frame-stacker
stacker = FrameStacker()
policy_net = PolicyNetwork(lr=learning_rate).to(device)

#some lists to write the values, if you want to do some in-notebook plotting.
num_steps = []
avg_numsteps = []
all_rewards = []

#we leverage this cell to write our graph to TensorBoard.
writer.add_graph(policy_net, stacker.stack(torch.zeros(84, 84)).unsqueeze(0).to(device))

#### We run our train steps now

While it runs you can watch its actions being taken on the game windows and see its learning metrics on the TensorBoard window.

In [ ]:
for episode in range(EPISODES):
    game.new_episode()
    state = game.get_state().screen_buffer
    
    state = stacker.stack(state)
    log_probs = []
    rewards = []
    
    done = False
    steps = 0
    
    while True:
        action_idx, log_prob = policy_net.get_action(state)
        action = doom_actions[action_idx]
        reward = game.make_action(action)
        done = game.is_episode_finished()
        rewards.append(reward)
        log_probs.append(log_prob)
        
        steps += 1
        
        if done:
            stacker.reset()
            break
            
        new_state = game.get_state().screen_buffer
        state = stacker.stack(new_state)
        
    writer.add_scalar("steps", steps, episode)
    
    update_policy(policy_net, rewards, log_probs)
    num_steps.append(steps)
    writer.add_scalar("avg_steps", np.mean(num_steps[-10:]), episode)
    
    avg_numsteps.append(np.mean(num_steps[-10:]))
    all_rewards.append(np.sum(rewards))
    print("Episode: {}, total_reward: {}, length: {}".format(episode+1, np.sum(rewards), steps))

Episode: 1, total_reward: 476.0, length: 576
Episode: 2, total_reward: 444.0, length: 544
Episode: 3, total_reward: 540.0, length: 640
Episode: 4, total_reward: 284.0, length: 384
Episode: 5, total_reward: 284.0, length: 384
Episode: 6, total_reward: 412.0, length: 512
Episode: 7, total_reward: 476.0, length: 576
Episode: 8, total_reward: 700.0, length: 800
Episode: 9, total_reward: 1980.0, length: 2080
Episode: 10, total_reward: 988.0, length: 1088
Episode: 11, total_reward: 892.0, length: 992
Episode: 12, total_reward: 604.0, length: 704
Episode: 13, total_reward: 284.0, length: 384
Episode: 14, total_reward: 764.0, length: 864
Episode: 15, total_reward: 764.0, length: 864
Episode: 16, total_reward: 572.0, length: 672
Episode: 17, total_reward: 508.0, length: 608
Episode: 18, total_reward: 284.0, length: 384
Episode: 19, total_reward: 1148.0, length: 1248
Episode: 20, total_reward: 668.0, length: 768
Episode: 21, total_reward: 668.0, length: 768
Episode: 22, total_reward: 1180.0, len

In [ ]:
#we do some in-notebook plotting if you want it.
plt.plot(num_steps)
plt.plot(avg_numsteps)
plt.xlabel('Episode')
plt.show()